In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from config import PGHOST, PGDATABASE, PGUSER, PGPASSWORD

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM

In [2]:
# Define a function to connect to AWS database instance

def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+PGHOST +" port="+ "5432" +" dbname="+PGDATABASE +" user=" + PGUSER \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [3]:
conn, cursor = connect()

Connected!


In [4]:
# Get the data, read SQL query into data frame
bb_data = pd.read_sql_query("SELECT * FROM blackberry", con=conn)
bb_df = pd.DataFrame(bb_data)
bb_df.head()

,Date,open_price,high_price,low_price,close_price,adj_close,volume
0,1999-02-04,2.145833,2.166667,1.895833,1.924479,1.924479,16788600
1,1999-02-05,1.929688,1.947917,1.822917,1.833333,1.833333,3053400
2,1999-02-08,1.854167,1.927083,1.783854,1.812500,1.812500,1548000
3,1999-02-09,1.822917,1.833333,1.656250,1.666667,1.666667,3501600
4,1999-02-10,1.708333,1.708333,1.604167,1.677083,1.677083,1597200


In [5]:
# Get the data, read SQL query into data frame
# gg_data = pd.read_sql_query("SELECT * FROM google", con=conn)
# gg_df = pd.DataFrame(gg_data)
# gg_df.head()

In [6]:
# Get the data, read SQL query into data frame
# nf_data = pd.read_sql_query("SELECT * FROM netflix", con=conn)
# nf_df = pd.DataFrame(nf_data)
# nf_df.head()

In [7]:
# Check the rows and columns of dfs
# print(bb_df.shape)
# print(gg_df.shape)
# print(nf_df.shape)

In [8]:
# Check if there is null values
# bb_df.isnull().sum()

In [9]:
# Check the data types in df
# bb_df.info()

In [10]:
# Change [Date] data type to data time

bb_df['Date'] = pd.to_datetime(bb_df.Date)
# gg_df['Date'] = pd.to_datetime(gg_df.Date)
# nf_df['Date'] = pd.to_datetime(nf_df.Date)

In [11]:
# bb_df.info()

In [12]:
# Delete 'Adj Close' column and check df

bb_df = bb_df.drop('adj_close', axis=1)
# gg_df = gg_df.drop('adj_close', axis=1)
# nf_df = nf_df.drop('adj_close', axis=1)

In [13]:
# bb_df.set_index('Date')
# gg_df.set_index('Date')
# nf_df.set_index('Date')

In [14]:
# Vistulize the stock price trend by using 'Close' column
# plt.plot(bb_df['Date'], bb_df['close_price'], label='Close')
# plt.xlabel('Date')
# plt.ylabel('Price')
# plt.title('BB History Stock Price')
# plt.grid()
# plt.legend()
# plt.show()

In [15]:
# Vistulize the stock price trend by using 'Close' column
# plt.plot(gg_df['Date'], gg_df['close_price'], label='Close')
# plt.xlabel('Date')
# plt.ylabel('Price')
# plt.title('Google History Stock Price')
# plt.grid()
# plt.legend()
# plt.show()

In [16]:
# Vistulize the stock price trend by using 'Close' column
# plt.plot(nf_df['Date'], nf_df['close_price'], label='Close')
# plt.xlabel('Date')
# plt.ylabel('Price')
# plt.title('Netflix History Stock Price')
# plt.grid()
# plt.legend()
# plt.show()

# Multiple Linear Regression Model

In [17]:
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split

# # Split our preprocessed data into our features and target arrays

# X = bb_df[['open_price','high_price','low_price','volume']].values
# y = bb_df['close_price'].values

# # Split the preprocessed data into a training and testing dataset

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# # Check the shape of X_train and X_test

# print(X_train.shape)
# print(X_test.shape)

# # Build linear reg model

# lr_model = LinearRegression()
# lr_model.fit(X_train, y_train)

# # Check the model score
# #lr_model.score(X_train, y_train)

# # Make stock price prediction

# prediction = lr_model.predict(X_test)

# print(prediction.shape)
# print(prediction)

# # Check R sqr value
# from sklearn.metrics import r2_score
# print(r2_score(y_test, prediction))

# # Compare y_test and our prediction value

# pred_df = pd.DataFrame({'Actual Price':y_test, 'Predicted Price':prediction})

# pred_df.head()

In [18]:
# Plot the actual and predicted price
# from the above data frame, we can see the difference between 'actual price' and 'predicted price' is tiny
# if using line graph, the two lines will be overlapped to each other.
# so we choose to use bar graph to show the difference of first 20 rows of data frame

# fig_df = pred_df.loc[pred_df.index<20]

# x = np.arange(len(fig_df['Actual Price']))  # the label locations
# width = 0.35  # the width of the bars
# fig, ax = plt.subplots()
# rects1 = ax.bar(x - width/2, fig_df['Actual Price'], width, label='Actual')
# rects2 = ax.bar(x + width/2, fig_df['Predicted Price'], width, label='Predicted')

# # Add some text for labels, title and custom x-axis tick labels, etc.
# ax.set_xlabel('Time')
# ax.set_ylabel('Stock Price')
# ax.set_title('Stock Price predicted by Linear Regression Model')
# ax.legend()
# fig.tight_layout()
# plt.show()

In [19]:
# print(lr_model.coef_)
# print(lr_model.intercept_)

In [20]:
# Calculate the MSE for our model
# The mean squared error (MSE) tells you how close a regression line is to a set of points
# return a non-negative floating point value (the best value is 0.0)

# from sklearn.metrics import mean_squared_error
# mean_squared_error(y_test, prediction)

# Using more popular LSTM model to predict BB stock price

In [21]:
bb_df = bb_df.loc[bb_df.index < 20]


In [22]:
data_len = round(len(bb_df) * 0.8) # 80% of data as train 

In [23]:
x = bb_df[['open_price','low_price','volume']]

y = pd.DataFrame(bb_df['close_price'])

In [24]:
x_train = x.loc[x.index < data_len]

y_train = y.loc[y.index < data_len]

In [25]:
x_test = x.loc[x.index >= data_len]

y_test = y.loc[y.index >= data_len]

In [26]:
# Choose "Closed" price column as our input variable
# X_df = bb_df[['open_price','low_price','volume']]
# y_df = pd.DataFrame(bb_df['close_price'])

# data_len = round(len(X_df) * 0.8)

# X_df_train = X_df.loc[X_df.index < data_len]
# X_df_test = X_df.loc[X_df.index >= data_len]

# print(X_df_train.head())
# print()
# print(X_df_test.head())
# print()
# print(y_df.head())

In [27]:
# Scale the training and testing data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data_train_open = pd.DataFrame.from_records(scaler.fit_transform(x_train['open_price'].values.reshape(-1,1)))
scaled_data_train_low = pd.DataFrame.from_records(scaler.fit_transform(x_train['low_price'].values.reshape(-1,1)))
scaled_data_train_volume = pd.DataFrame.from_records(scaler.fit_transform(x_train['volume'].values.reshape(-1,1)))
scaled_data_train_close = scaler.fit_transform(y_train.values.reshape(-1,1))
#scaled_data_test = scaler.fit_transform(X_df_test.values.reshape(-1,1))

In [42]:
result = pd.concat([scaled_data_train_open, scaled_data_train_low, scaled_data_train_volume], axis=1, join="inner")

x_train = result.set_axis(['open', 'low', 'vol'], axis=1, inplace=False) 

x_train.shape

(16, 3)

In [43]:
scaled_data_train_close.shape

(16, 1)

In [47]:
x_train = np.array(x_train)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [29]:
# Use the previous 30 days 'close' price to predict the 31st day's price
previous_days = 30

In [30]:
# Splite data into X_train and y_train
# X_train = []
# y_train = X_df_train[previous_days:]
# X_test = []
# y_test = X_df_test[previous_days:]

# for i in range(previous_days, len(scaled_data_train)):
#     X_train.append(scaled_data_train[i-previous_days:i, :])
#     #y_train.append(scaled_data_train[i, 0])
    
# for i in range(previous_days, len(scaled_data_test)):
#     X_test.append(scaled_data_test[i-previous_days:i, 0])

In [31]:
# print(X_train)

In [32]:
# print(X_test)

In [33]:
# Convert the X_train and y_train to numpy arrays
# X_train, y_train, X_test = np.array(X_train), np.array(y_train), np.array(X_test)

# print(X_train.shape)
# print(y_train.shape)
# print(X_test.shape)

In [34]:
# Reshape the X_train since LSTM needs 3-dimensional inputs
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# print(X_train.shape)
# print(X_test.shape)

In [48]:
# Build the LSTM model
model = Sequential()
# Add LSTM layer
model.add(LSTM(units = 50, return_sequences=True, input_shape = (x_train.shape[1],3)))
model.add(LSTM(units = 50, return_sequences=True))
model.add(LSTM(units = 50))
# prediction output layer
model.add(Dense(units=1))

In [50]:
# compile and train the data
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size =1, epochs=5)

Epoch 1/5


ValueError: in user code:

    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ivorf\anaconda3\envs

In [51]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 3, 50)             10800     
                                                                 
 lstm_7 (LSTM)               (None, 3, 50)             20200     
                                                                 
 lstm_8 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 51,251
Trainable params: 51,251
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Predict the data

prediction = model.predict(x_test)
prediction = scaler.inverse_transform(prediction)

print(len(prediction))

ValueError: in user code:

    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Users\ivorf\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ivorf\anaconda

In [ ]:

y_test['pred'] = prediction
y_test.head()


In [ ]:
# plot test predictions


plt.figure(figsize=(12, 8))
plt.plot(close_df_train['Date'],close_df_train)
plt.plot(y_test[['close_price','pred']])
plt.legend(['Train', 'Val', 'Predictions'])
plt.xlabel('Date')
plt.ylabel('Stock Price')

plt.title("Stock Price predicted by LSTM Model")
plt.grid()
plt.legend()
plt.show()

# Using LSTM model to predict GOOGLE stock price

In [ ]:
# Choose "Closed" price column as our input variable
close_df= gg_df['close_price']

data_len = round(len(close_df) * 0.8)

close_df_train = close_df.loc[close_df.index < data_len]
close_df_test = close_df.loc[close_df.index >= data_len]

print(close_df_train.head())
print()
print(close_df_test.head())

In [ ]:
# Scale the training data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(close_df_train.values.reshape(-1,1))
scaled_data.shape

In [ ]:
# Use the previous 30 days 'close' price to predict the 31st day's price
previous_days = 30

In [ ]:
# Splite data into X_train and y_train
X_train = []
y_train = []

for i in range(previous_days, len(scaled_data)):
    X_train.append(scaled_data[i-previous_days:i, 0])
    y_train.append(scaled_data[i, 0])

In [ ]:
# Convert the X_train and y_train to numpy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train.shape)
print(y_train.shape)

In [ ]:
# Reshape the X_train since LSTM needs 3-dimensional inputs
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

In [ ]:
# Build the LSTM model
model = Sequential()
# Add LSTM layer
model.add(LSTM(units = 50, return_sequences=True, input_shape = (X_train.shape[1],1)))
model.add(LSTM(units = 50, return_sequences=True))
model.add(LSTM(units = 50))
# prediction output layer
model.add(Dense(units=1))

In [ ]:
# compile and train the data
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size =32, epochs=20)

In [ ]:
model.summary()

In [ ]:
#### Test model ####

# Scale the testing data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_test_data = scaler.fit_transform(close_df_test.values.reshape(-1,1))
scaled_test_data.shape

In [ ]:
# Create X_test and y_test
X_test = []
y_test = []

for i in range(previous_days, len(scaled_test_data)):
    X_test.append(scaled_test_data[i-previous_days:i, 0])
    y_test.append(scaled_test_data[i, 0])

In [ ]:
# Convert the X_test to numpy arrays
X_test = np.array(X_test)
print(X_test.shape)

In [ ]:
# reshape
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print(X_test.shape)

In [ ]:
# Predict the data

prediction = model.predict(X_test)
prediction = scaler.inverse_transform(prediction)

In [ ]:
# plot test predictions

plt.figure(figsize=(12, 8))
plt.plot(y_test, color='red', label='Actual')
plt.plot(prediction, color='green', label='Predicted')
plt.xlabel('Date')
plt.ylabel('Stock Price')

plt.title("Stock Price predicted by LSTM Model")
plt.grid()
plt.legend()
plt.show()